In [2]:
import math
import collections
import numpy as onp
import itertools
from mxnet import np, gluon

In [14]:
f = open("data/ictrain", "r")
ex = f.read()
emission_score = dict(collections.Counter(ex.split()))
emission_score.pop('###/###')
ex_sent = ex.split('###/###')
sentences = []
for sent in ex_sent:
    sentences.append(sent.split())
transition_score = {}
tag_dict = {}
for x in emission_score:
    if(x[2] in tag_dict):
        tag_dict[x[2]] = tag_dict[x[2]] + emission_score[x]
    else:
        tag_dict[x[2]] = emission_score[x]
print(emission_score)
print(tag_dict)
for x in emission_score:
    emission_score[x] = emission_score[x]/tag_dict[x[2]]
#print(tag_dict)
vocabulary = []
for key in tag_dict.keys():
    k = key
    vocabulary.append(k)
vocabulary.append('$')
vocabulary.append('#')
print(vocabulary)
print(emission_score)
sentences.pop(len(sentences)-1)
for sent in sentences:
    sent.insert(0,'$/$')
    sent.append('#/#')
#print(sentences)
transition_score = {}
for i in range(0,len(sentences)):
    for j in range(1, len(sentences[i])):
        curr_tag = sentences[i][j-1][2]+sentences[i][j][2]
        if(curr_tag in transition_score):
            transition_score[curr_tag] = transition_score[curr_tag] + 1
        else:
            transition_score[curr_tag] = 1
for i in vocabulary:
    for j in vocabulary:
        curr_tag = i+j
        if(curr_tag in transition_score):
            transition_score[curr_tag] = transition_score[curr_tag] + 1
        else:
            transition_score[curr_tag] = 1
tag_tag = {}
for x in transition_score:
    if(x[0] in tag_tag):
        tag_tag[x[0]] += transition_score[x]
    else:
        tag_tag[x[0]] = transition_score[x]
for x in transition_score:
    transition_score[x] /= tag_tag[x[0]]

print(transition_score)

{'1/C': 14, '2/C': 4, '3/H': 14, '1/H': 2, '2/H': 4, '3/C': 2}
{'C': 20, 'H': 20}
['C', 'H', '$', '#']
{'1/C': 0.7, '2/C': 0.2, '3/H': 0.7, '1/H': 0.1, '2/H': 0.2, '3/C': 0.1}
{'$C': 0.375, 'CC': 0.7083333333333334, 'CH': 0.125, 'H#': 0.125, '$H': 0.375, 'HH': 0.7083333333333334, 'HC': 0.125, 'C#': 0.125, 'C$': 0.041666666666666664, 'H$': 0.041666666666666664, '$$': 0.125, '$#': 0.125, '#C': 0.25, '#H': 0.25, '#$': 0.25, '##': 0.25}


In [15]:
f = open("data/ictest", "r")
ex = f.read()
#print(ex)
ex = ex.split()
ex.remove('###/###')
#ex = sentences[0]
#print(ex)
result = np.zeros((len(ex)))
for i in range(0,len(ex)):
    z = ex[i][2]
    result[i] = vocabulary.index(z)
#print(result)
pi = np.zeros((len(vocabulary),len(ex)+1))
#print(pi.shape)
#print(test)
"""
for i in range(0,len(vocabulary)):
    combo = ex[0][0]+'/'+vocabulary[i]
    pi[i,0] = emission_score[combo]
for i in range(1,len(ex)):
    for j in range(0,len(vocabulary)):
        combo = ex[i][0]+'/'+vocabulary[j]
        pi[j,i] = emission_score[combo]
        max_score = 0
        #print(combo)
        for k in range(0,len(vocabulary)):
            combo = vocabulary[k]+vocabulary[j]
            max_score = max(transition_score[combo]*pi[k,i-1], max_score)
            #print(combo)
            #print(max_score)
        pi[j,i] *= max_score
        #print(pi[j,i])
        #print()
"""
for i in range(0,len(vocabulary)):
    combo = '$'+vocabulary[i]
    pi[i,0] = transition_score[combo]
for i in range(1,len(ex)+1):
    for j in range(0,len(vocabulary)-2):
        combo = ex[i-1][0]+'/'+vocabulary[j]
        pi[j,i] = emission_score[combo]
        max_score = 0
        for k in range(0,len(vocabulary)):
            combo = vocabulary[k]+vocabulary[j]
            max_score = max(transition_score[combo]*pi[k,i-1], max_score)
        pi[j,i] *= max_score
    
    
    
x = np.argmax(pi,0)
#print(pi)
error = 0
for i in range(0,len(result)):
    if(result[i] - x[i]!=0.):
        error+=1
#print(x)
accuracy = (1 - (error/len(result)))*100
print(accuracy)
#for i in range(0,len(ex)):
    #print(result[i],x[i],result[i] - x[i])


75.75757575757575
